# Movies
Author: [Oleg Ovcharuk](https://github.com/vgvoleg)  
Dataset: [the-movies-dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)



## Homework №1
Предсказание оценки фильма по данным

In [1]:
import pandas as pd
import json
from ast import literal_eval

### Cleaning Datasets

Некоторые утильные функции, которые пригодятся в дальшейнем

In [2]:
def safe_lambda(dataset, column, lmbd):
    for value in dataset[column]:
        try:
            value = lmbd(value)
        except:
            print("Incorrect value {} for column {}".format(value, column))
            dataset = dataset.drop(dataset[dataset[column] == value].index)
    dataset[column] = dataset[column].apply(lmbd)
    return dataset

def parse_json(s):
    # на самом деле тут повсеместно встречается невалидный 
    # синтаксис json'a, функция добавлена для улучшения читаемости
    return literal_eval(s)

Датасет разбит на 3 части:
1. movies_metadata - метаинформация о фильме: название, длительность, жанры, бюджет, средняя оценка
2. keywords - ключевые слова сюжета
3. credits - информация о людях, участвующих при создании фильма: актеры, режиссер и прочее  

Данные переплетаются между таблицами путем наличия id фильма.

In [3]:
m_meta = pd.read_csv("./the-movies-dataset/movies_metadata.csv", low_memory=False, sep=',')
m_keywords = pd.read_csv("./the-movies-dataset/keywords.csv", low_memory=False, sep=',')
m_credits = pd.read_csv("./the-movies-dataset/credits.csv", low_memory=False, sep=',')

####  Dataset m_meta

In [4]:
print(m_meta.shape)
print(m_meta.columns)

(45466, 24)
Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


В данном датасете очень много неинформативных столбцов: избавимся от них.

In [5]:
m_meta = m_meta.drop([
    'adult', 'belongs_to_collection', 'homepage', 'imdb_id', 'release_date',
    'budget', 'original_language', 'original_title', 'overview', 'popularity',
    'poster_path', 'production_companies', 'production_countries', 
    'revenue', 'spoken_languages', 'status', 'tagline', 'video'
], axis=1, errors='ignore')
print(m_meta.shape)
print(m_meta.columns)
m_meta.head()

(45466, 6)
Index(['genres', 'id', 'runtime', 'title', 'vote_average', 'vote_count'], dtype='object')


,genres,id,runtime,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,81.0,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,104.0,Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,101.0,Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,127.0,Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,106.0,Father of the Bride Part II,5.7,173.0


In [6]:
m_meta.dtypes

genres           object
id               object
runtime         float64
title            object
vote_average    float64
vote_count      float64
dtype: object

In [7]:
# я не понимаю от чего это зависит, но через раз в столбик id 
# попадают левые значения, поэтому ниже - страховка
m_meta = safe_lambda(m_meta, 'id', int)
m_meta.dtypes

Incorrect value 1997-08-20 for column id
Incorrect value 2012-09-29 for column id
Incorrect value 2014-01-01 for column id


genres           object
id                int64
runtime         float64
title            object
vote_average    float64
vote_count      float64
dtype: object

Рассмотрим колонку *genres*:

In [8]:
m_meta.genres.head()

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

Данные преставлены в плохо читаемом формате, более того - это одна большая строка, требующая обработки.

In [9]:
m_meta.genres = m_meta.genres.apply(parse_json).apply(lambda x: [genre['name'] for genre in x])
m_meta.genres.head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

#### Dataset `m_keywords`

In [10]:
print(m_keywords.shape)
print(m_keywords.columns)

(46419, 2)
Index(['id', 'keywords'], dtype='object')


In [11]:
m_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [12]:
# на всякий случай провернем и тут страховку
m_keywords = safe_lambda(m_keywords, 'id', int)
m_keywords.dtypes

id           int64
keywords    object
dtype: object

В столбце *keywords* наблюдается похожая ситуация, как с колонкой *genres*. Обработаем аналогичным образом.

In [13]:
m_keywords.keywords = m_keywords.keywords.apply(parse_json).apply(lambda x: [kw['name'] for kw in x])
m_keywords.head()

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


#### Dataset `m_credits`

In [14]:
print(m_credits.shape)
print(m_credits.columns)

(45476, 3)
Index(['cast', 'crew', 'id'], dtype='object')


In [15]:
m_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Слооожно. Изучим содержимое полей более детально.

In [16]:
print(m_credits.cast[1])

[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.jpg'}, {'cast_id': 8, 'character': 'Samuel Alan Parrish / Van Pelt', 'credit_id': '52fe44bfc3a36847f80a7c99', 'gender': 2, 'id': 8537, 'name': 'Jonathan Hyde', 'order': 1, 'profile_path': '/7il5D76vx6QVRVlpVvBPEC40MBi.jpg'}, {'cast_id': 2, 'character': 'Judy Sheperd', 'credit_id': '52fe44bfc3a36847f80a7c77', 'gender': 1, 'id': 205, 'name': 'Kirsten Dunst', 'order': 2, 'profile_path': '/wBXvh6PJd0IUVNpvatPC1kzuHtm.jpg'}, {'cast_id': 24, 'character': 'Peter Shepherd', 'credit_id': '52fe44c0c3a36847f80a7ce7', 'gender': 0, 'id': 145151, 'name': 'Bradley Pierce', 'order': 3, 'profile_path': '/j6iW0vVA23GQniAPSYI6mi4hiEW.jpg'}, {'cast_id': 10, 'character': 'Sarah Whittle', 'credit_id': '52fe44bfc3a36847f80a7c9d', 'gender': 1, 'id': 5149, 'name': 'Bonnie Hunt', 'order': 4, 'profile_path': '/7spiVQwmr8

В столбце *cast* указан весь актерский состав. Действительно ли важны для изучения оценки фильма все актеры? Вряд ли. По логике, нужно брать в расчет только тех актеров, которые внесли наибольший вклад в фильм, то есть главные роли (как правило, это те люди, из-за которых зрители вообще покупают билет в кино). И как же нам повезло, что в данном столбике значения указаны в отсортированном порядке! Для рассмотрения будем брать первых 4 (если столько имеется) актеров, указанных в каст-листе.

In [17]:
def cut_main_actors(cast_list):
    if len(cast_list) > 4:
        cast_list = cast_list[:4]
    return [actor['name'] for actor in cast_list]
m_credits.cast = m_credits.cast.apply(parse_json).apply(cut_main_actors)
print(m_credits.cast[1])

['Robin Williams', 'Jonathan Hyde', 'Kirsten Dunst', 'Bradley Pierce']


Рассмотрим *crew*

In [18]:
print(m_credits.crew[1])

[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, 'job': 'Executive Producer', 'name': 'Larry J. Franco', 'profile_path': None}, {'credit_id': '52fe44bfc3a36847f80a7c89', 'department': 'Writing', 'gender': 2, 'id': 876, 'job': 'Screenplay', 'name': 'Jonathan Hensleigh', 'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cdd', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'name': 'James Horner', 'profile_path': '/oLOtXxXsYk8X4qq0ud4xVypXudi.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7c7d', 'department': 'Directing', 'gender': 2, 'id': 4945, 'job': 'Director', 'name': 'Joe Johnston', 'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cd7', 'department': 'Editing', 'gender': 2, 'id': 4951, 'job': 'Editor', 'name': 'Robert Dalva', 'profile_path': None}, {'credit_id': '573523bec3a368025100062c', 'department': 'Production', 'gender': 0, 'i

И снова слишком много информации. В съемочной команде очень много ролей, однако одна из основополагающих (опять же, эта роль порой продает билеты в кино) - это режиссер. Если Дэвид Финчер снимает глубокие и интересные фильмы, то это в какой-то момент становится фактом, и любой его новый фильм будет встречаться с ажиотажем и овациями. Это достаточно хороший признак для использования при определении оценки фильма.

In [19]:
def cut_director(crew):
    for role in crew:
        if role['job'] == 'Director':
            return role['name']
    return None
m_credits.crew = m_credits.crew.apply(parse_json).apply(cut_director)

# ну и заодно переименуем столбец
m_credits.rename(columns={'crew': 'director'}, inplace=True)
print(m_credits.director[1])

Joe Johnston


In [20]:
m_credits.dtypes

cast        object
director    object
id           int64
dtype: object

In [21]:
# очередная страховка
m_credits = safe_lambda(m_credits, 'id', int)

#### Merging datasets

Для дальнейшего удобства работы объединим информацию в один DataFrame:

In [22]:
# Сливаем данные в один Dataframe
movies = m_meta.copy()
movies = movies.merge(m_credits, on='id')
movies = movies.merge(m_keywords, on='id')
print(movies.shape)
print(movies.columns)

(46628, 9)
Index(['genres', 'id', 'runtime', 'title', 'vote_average', 'vote_count',
       'cast', 'director', 'keywords'],
      dtype='object')


In [23]:
print(movies.dtypes)

genres           object
id                int64
runtime         float64
title            object
vote_average    float64
vote_count      float64
cast             object
director         object
keywords         object
dtype: object


In [24]:
movies.head()

,genres,id,runtime,title,vote_average,vote_count,cast,director,keywords
0,"[Animation, Comedy, Family]",862,81.0,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]",John Lasseter,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[Adventure, Fantasy, Family]",8844,104.0,Jumanji,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,"[board game, disappearance, based on children'..."
2,"[Romance, Comedy]",15602,101.0,Grumpier Old Men,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,"[fishing, best friend, duringcreditsstinger, o..."
3,"[Comedy, Drama, Romance]",31357,127.0,Waiting to Exhale,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,"[based on novel, interracial relationship, sin..."
4,[Comedy],11862,106.0,Father of the Bride Part II,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,"[baby, midlife crisis, confidence, aging, daug..."


In [25]:
# поменяем местами столбцы для удобства
movies = movies[['id', 'title', 'runtime', 'director', 'cast', 'keywords', 'vote_count', 'vote_average']]